In [236]:
# Import libraries
from pyspark.mllib.recommendation import *
import random
from operator import *
from collections import defaultdict

#import libraries
from pyspark import SparkContext
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession ,Row
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType,StructField,IntegerType
import matplotlib.pyplot as plt

appName="Collaborative Filtering with PySpark"

#initialize the spark session
spark = SparkSession.builder.appName(appName).getOrCreate()

#get sparkcontext from the sparksession
sc = spark.sparkContext
sqlContext = SQLContext(sc)
path = "train_triplets.txt"
data = spark.read.csv(path, sep=r'\t', header=False)

/Users/jayantakumar_1/PycharmProjects/musicRecommendation/venv/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [237]:
data.show(n=3)

+--------------------+------------------+---+
|                 _c0|               _c1|_c2|
+--------------------+------------------+---+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|  1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|  1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|  2|
+--------------------+------------------+---+
only showing top 3 rows



In [238]:
data = data.withColumnRenamed("_c0","user").withColumnRenamed("_c1","song").withColumnRenamed("_c2","playCount")

In [239]:
data.printSchema()

root
 |-- user: string (nullable = true)
 |-- song: string (nullable = true)
 |-- playCount: string (nullable = true)



In [240]:
data.show(n=3)

+--------------------+------------------+---------+
|                user|              song|playCount|
+--------------------+------------------+---------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|
+--------------------+------------------+---------+
only showing top 3 rows



In [241]:
import pyspark.sql.functions as F

df2= data.withColumn("userId", F.hash(col("user")))

df2= df2.withColumn("songId", F.hash(col("song")))
# userId_change = data.select('user').distinct().select('user', F.monotonically_increasing_id().alias('userid'))
# songId_change = data.select('song').distinct().select('song', F.monotonically_increasing_id().alias('songid'))
# df2 = data.join(userId_change, 'user').join(songId_change, 'song')
df2 = df2.withColumn("playCount",col("playCount").cast(IntegerType()))
df2.show()
df2 = df2.limit(100000)

+--------------------+------------------+---------+----------+-----------+
|                user|              song|playCount|    userId|     songId|
+--------------------+------------------+---------+----------+-----------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|1365117428| 1315780877|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|1365117428|-1623759929|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|1365117428|-1218290021|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|1365117428|-1227648141|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|1365117428| 2054460487|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|        1|1365117428| 1741539561|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|        2|1365117428| 1352462404|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|        1|1365117428|-1539040688|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|        1|1365117428| 1734414350|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|        1|1365117428| 1593338752|
|b80344d063b5ccb32...|SOB

In [242]:
(training, test) = df2.randomSplit([0.7, 0.3],seed = 20)

In [53]:
# rank=20,maxIter=10, regParam=0.01,
als = ALS(
          userCol="userId", itemCol="songId", ratingCol="playCount",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)
# Import the requisite packages
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 15, 25, 100]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse",
           labelCol="playCount",
           predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
#Fit cross validator to the 'train' dataset
model = cv.fit(training)
#Extract best model from the cv model above
best_model = model.bestModel
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/jayantakumar_1/PycharmProjects/musicRecommendation/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/jayantakumar_1/PycharmProjects/musicRecommendation/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [67]:
als = ALS(rank=100,maxIter=10,regParam=0.15,
          userCol="userId", itemCol="songId", ratingCol="playCount",
          coldStartStrategy="drop",
          implicitPrefs=True)
best_model = als.fit(training)
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

6.93831915864255


In [68]:
# Generate n Recommendations for all users
recommendations = best_model.recommendForAllUsers(5)
recommendations.show()

+-----------+--------------------+
|     userId|     recommendations|
+-----------+--------------------+
|-2125095252|[{618915878, 0.39...|
|-2100266772|[{-1345472587, 0....|
|-2088470684|[{1662567676, 0.0...|
|-2087947144|[{-1882962528, 0....|
|-2081511422|[{-2053570789, 0....|
|-2078707018|[{-1297511273, 1....|
|-2078322363|[{1307306596, 0.0...|
|-2052897808|[{2117399503, 0.0...|
|-2037466548|[{1877036441, 0.8...|
|-2029966273|[{-1345743536, 0....|
|-2010071424|[{-1810233123, 0....|
|-1983755226|[{-118608652, 0.1...|
|-1964357835|[{-968376098, 0.5...|
|-1949914158|[{172050612, 0.04...|
|-1933481207|[{-1325963706, 0....|
|-1925532506|[{-1687914835, 0....|
|-1920839507|[{1838416373, 0.3...|
|-1915006186|[{-474679864, 0.9...|
|-1903552246|[{474833235, 0.64...|
|-1898771103|[{-1002846672, 0....|
+-----------+--------------------+
only showing top 20 rows



In [69]:
from pyspark.sql.functions import explode
nrecommendations = recommendations\
    .withColumn("rec_exp",  explode("recommendations"))\
    .select('userId', col("rec_exp.songId"), col("rec_exp.rating"))
nrecommendations.limit(10).show()


+-----------+-----------+----------+
|     userId|     songId|    rating|
+-----------+-----------+----------+
|-2125095252|  618915878|  0.397131|
|-2125095252| -344769466|0.32552007|
|-2125095252|-1092166790|  0.215502|
|-2125095252|  330169866| 0.2006451|
|-2125095252| 1580252322|0.18503353|
|-2100266772|-1345472587|  0.646069|
|-2100266772|  634070714|0.63387823|
|-2100266772| 1125837201| 0.6048141|
|-2100266772| 1877036441|  0.589782|
|-2100266772|  591135417| 0.5591793|
+-----------+-----------+----------+



In [71]:
path = "unique_tracks.txt"
tracks = spark.read.csv(path,sep="<SEP>")
tracks.show()

+------------------+------------------+--------------------+--------------------+
|               _c0|               _c1|                 _c2|                 _c3|
+------------------+------------------+--------------------+--------------------+
|TRMMMYQ128F932D901|SOQMMHC12AB0180CB8|    Faster Pussy cat|        Silent Night|
|TRMMMKD128F425225D|SOVFVAK12A8C1350D9|    Karkkiautomaatti|         Tanssi vaan|
|TRMMMRX128F93187D9|SOGTUKN12AB017F4F1|      Hudson Mohawke|   No One Could Ever|
|TRMMMCH128F425532C|SOBNYVR12A8C13558C|         Yerba Brava|       Si Vos Querés|
|TRMMMWA128F426B589|SOHSBXH12A8C13B0DF|          Der Mystic|    Tangle Of Aspens|
|TRMMMXN128F42936A5|SOZVAPQ12A8C13B63C|    David Montgomery|Symphony No. 1 G ...|
|TRMMMLR128F1494097|SOQVRHI12A6D4FB2D7|  Sasha / Turbulence|    We Have Got Love|
|TRMMMBB12903CB7D21|SOEYRFT12AB018936C|          Kris Kross|   2 Da Beat Ch'yall|
|TRMMMHY12903CB53F1|SOPMIYT12A6D4F851E|        Joseph Locke|             Goodbye|
|TRMMMML128F4280

In [72]:
tracks = tracks.withColumnRenamed("_c0","tid").withColumnRenamed("_c1","song").withColumnRenamed("_c2","ArtistName").withColumnRenamed("_c3","SongTitle")
tracks= tracks.withColumn("songId", F.hash(col("song")))

In [73]:
tracks.limit(5).show()

+------------------+------------------+----------------+-----------------+----------+
|               tid|              song|      ArtistName|        SongTitle|    songId|
+------------------+------------------+----------------+-----------------+----------+
|TRMMMYQ128F932D901|SOQMMHC12AB0180CB8|Faster Pussy cat|     Silent Night|-561391088|
|TRMMMKD128F425225D|SOVFVAK12A8C1350D9|Karkkiautomaatti|      Tanssi vaan|-147901473|
|TRMMMRX128F93187D9|SOGTUKN12AB017F4F1|  Hudson Mohawke|No One Could Ever| 547521741|
|TRMMMCH128F425532C|SOBNYVR12A8C13558C|     Yerba Brava|    Si Vos Querés|-624908617|
|TRMMMWA128F426B589|SOHSBXH12A8C13B0DF|      Der Mystic| Tangle Of Aspens|1651792394|
+------------------+------------------+----------------+-----------------+----------+



In [78]:
nrecommendations.join(tracks, on='songId').filter('userId = -2125095252').sort('rating',ascending=False).show()

+-----------+-----------+----------+------------------+------------------+--------------------+--------------------+
|     songId|     userId|    rating|               tid|              song|          ArtistName|           SongTitle|
+-----------+-----------+----------+------------------+------------------+--------------------+--------------------+
|  618915878|-2125095252|  0.397131|TRANPNT128F9306A7F|SOARTRF12AB0181F4D|     Efecto Mariposa|        Por quererte|
| -344769466|-2125095252|0.32552007|TRIRFDS128E079285B|SOXWTDX12A6701E141|   Eve / Truth Hurts|                What|
|-1092166790|-2125095252|  0.215502|TREJSDF12903CB0FDA|SOPSAEJ12A58A7DA0F|              Camila|              Besame|
|  330169866|-2125095252| 0.2006451|TRMHBNJ12903CF116E|SOXZSEH12AC468CABB|       Amity in fame|  Relegate the Judge|
| 1580252322|-2125095252|0.18503353|TRFILTB128EF34566C|SOEMWNK12A67AD8797|Talib Kweli / Pha...|Guerrilla Monsoon...|
+-----------+-----------+----------+------------------+---------

In [77]:

df2.join(tracks,on="songId").filter('userId = -2125095252').sort('playCount', ascending=False).select("songId","userId","playCount","songTitle",).limit(15).show()

+-----------+-----------+---------+--------------------+
|     songId|     userId|playCount|           songTitle|
+-----------+-----------+---------+--------------------+
|  618915878|-2125095252|       10|        Por quererte|
| -344769466|-2125095252|        6|                What|
|-1092166790|-2125095252|        6|              Besame|
| 1200624010|-2125095252|        6|    Solo Dejate Amar|
|-2024781075|-2125095252|        5|The Christmas Son...|
| 1580252322|-2125095252|        5|Guerrilla Monsoon...|
|-1407064031|-2125095252|        5|        Dile al amor|
|  623026281|-2125095252|        4|        Fruta Fresca|
|  -14492029|-2125095252|        3|Ojalá que llueva ...|
| -569916139|-2125095252|        3|   Fique Em Silencio|
| -681206166|-2125095252|        2|              Dímelo|
| 2090221806|-2125095252|        1|            Hey Mama|
| 1293937135|-2125095252|        1|            Waterloo|
|-1367100942|-2125095252|        1| Wherever I May Roam|
| -100236852|-2125095252|      

In [79]:
df2.limit(10).show()

+--------------------+------------------+---------+----------+-----------+
|                user|              song|playCount|    userId|     songId|
+--------------------+------------------+---------+----------+-----------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|1365117428| 1315780877|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|1365117428|-1623759929|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|1365117428|-1218290021|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|1365117428|-1227648141|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|1365117428| 2054460487|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|        1|1365117428| 1741539561|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|        2|1365117428| 1352462404|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|        1|1365117428|-1539040688|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|        1|1365117428| 1734414350|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|        1|1365117428| 1593338752|
+--------------------+---

In [152]:

temp = df2.groupby("song").avg("playCount")
temp2 = df2.join(temp,on="song")
temp2.limit(10).show()
temp2 = temp2.withColumn("rating",F.col("playCount")/F.col("avg(playCount)"))

+------------------+--------------------+---------+----------+-----------+------------------+
|              song|                user|playCount|    userId|     songId|    avg(playCount)|
+------------------+--------------------+---------+----------+-----------+------------------+
|SOAKIMP12A8C130995|b80344d063b5ccb32...|        1|1365117428| 1315780877|               3.0|
|SOAPDEY12A81C210A9|b80344d063b5ccb32...|        1|1365117428|-1623759929|               4.5|
|SOBBMDR12A8C13253B|b80344d063b5ccb32...|        2|1365117428|-1218290021|              23.0|
|SOBFNSP12AF72A0E22|b80344d063b5ccb32...|        1|1365117428|-1227648141|               1.0|
|SOBFOVM12A58A7D494|b80344d063b5ccb32...|        1|1365117428| 2054460487|               1.0|
|SOBNZDC12A6D4FC103|b80344d063b5ccb32...|        1|1365117428| 1741539561|               1.0|
|SOBSUJE12A6D4F8CF5|b80344d063b5ccb32...|        2|1365117428| 1352462404|               2.0|
|SOBVFZR12A6D4F8AE3|b80344d063b5ccb32...|        1|136511742

In [153]:
temp2.limit(10).show()

+------------------+--------------------+---------+----------+-----------+------------------+-------------------+
|              song|                user|playCount|    userId|     songId|    avg(playCount)|             rating|
+------------------+--------------------+---------+----------+-----------+------------------+-------------------+
|SOAKIMP12A8C130995|b80344d063b5ccb32...|        1|1365117428| 1315780877|               3.0| 0.3333333333333333|
|SOAPDEY12A81C210A9|b80344d063b5ccb32...|        1|1365117428|-1623759929|               4.5| 0.2222222222222222|
|SOBBMDR12A8C13253B|b80344d063b5ccb32...|        2|1365117428|-1218290021|              23.0|0.08695652173913043|
|SOBFNSP12AF72A0E22|b80344d063b5ccb32...|        1|1365117428|-1227648141|               1.0|                1.0|
|SOBFOVM12A58A7D494|b80344d063b5ccb32...|        1|1365117428| 2054460487|               1.0|                1.0|
|SOBNZDC12A6D4FC103|b80344d063b5ccb32...|        1|1365117428| 1741539561|              

In [154]:
(training, test) = temp2.randomSplit([0.7, 0.3],seed = 20)
# rank=20,maxIter=10, regParam=0.01,

als = ALS(rank=100,maxIter=10,regParam=0.15,
          userCol="userId", itemCol="songId", ratingCol="rating",
          coldStartStrategy="drop",
          implicitPrefs=True)
best_model = als.fit(training)
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

# Generate n Recommendations for all users
recommendations = best_model.recommendForAllUsers(10)
recommendations.show()

from pyspark.sql.functions import explode
nrecommendations = recommendations\
    .withColumn("rec_exp",  explode("recommendations"))\
    .select('userId', col("rec_exp.songId"), col("rec_exp.rating"))
nrecommendations.limit(10).show()


]

5.9520136293623604


+-----------+--------------------+
|     userId|     recommendations|
+-----------+--------------------+
|-2125095252|[{618915878, 0.11...|
|-2081511422|[{-88411611, 0.03...|
|-2052897808|[{2117399503, 0.0...|
|-2037466548|[{634070714, 0.49...|
|-2029966273|[{-18793230, 0.03...|
|-2010071424|[{-1567249441, 0....|
|-1983755226|[{-2031457285, 0....|
|-1949914158|[{1415779473, 0.0...|
|-1933481207|[{-1540553036, 0....|
|-1925532506|[{1087921302, 0.0...|
|-1920839507|[{-1659736386, 0....|
|-1915006186|[{-561024783, 0.3...|
|-1903552246|[{-639326731, 0.0...|
|-1898771103|[{-892686094, 0.0...|
|-1864685400|[{1998285003, 0.1...|
|-1854769469|[{1935301280, 0.0...|
|-1843310532|[{1336006992, 0.0...|
|-1842936693|[{-1938766237, 0....|
|-1822857953|[{1650148391, 0.3...|
|-1818293155|[{-1846845641, 0....|
+-----------+--------------------+
only showing top 20 rows



+-----------+-----------+-----------+
|     userId|     songId|     rating|
+-----------+-----------+-----------+
|-2125095252|  618915878| 0.11344152|
|-2125095252| -344769466| 0.09309165|
|-2125095252|-1375191242| 0.08036793|
|-2125095252|-1407064031| 0.07198297|
|-2125095252|-1092166790| 0.07153061|
|-2125095252| -721871990| 0.06865166|
|-2125095252|  -74177708| 0.06548717|
|-2125095252|-1681840289| 0.06399172|
|-2125095252|-1065877168|0.060597338|
|-2125095252|-1269742185|  0.0583505|
+-----------+-----------+-----------+



In [155]:
nrecommendations.join(tracks, on='songId').filter('userId = -2125095252').sort('rating',ascending=False).select("songId","userId","rating","songTitle",).show()

+-----------+-----------+-----------+--------------------+
|     songId|     userId|     rating|           songTitle|
+-----------+-----------+-----------+--------------------+
|  618915878|-2125095252| 0.11344152|        Por quererte|
| -344769466|-2125095252| 0.09309165|                What|
|-1375191242|-2125095252| 0.08036793|Don't Cry (Original)|
|-1407064031|-2125095252| 0.07198297|        Dile al amor|
|-1092166790|-2125095252| 0.07153061|              Besame|
| -721871990|-2125095252| 0.06865166|   Siguiendo La Luna|
|  -74177708|-2125095252| 0.06548717|      Y Hubo Alguien|
|-1681840289|-2125095252| 0.06399172|       Paradise City|
|-1065877168|-2125095252|0.060597338|    Live And Let Die|
|-1065877168|-2125095252|0.060597338|    Live And Let Die|
|-1269742185|-2125095252|  0.0583505|            Abrazame|
+-----------+-----------+-----------+--------------------+



In [156]:
temp2.join(tracks,on="songId").filter('userId = -2125095252').sort('playCount', ascending=False).select("songId","userId","playCount","songTitle",).limit(20).show()

+-----------+-----------+---------+--------------------+
|     songId|     userId|playCount|           songTitle|
+-----------+-----------+---------+--------------------+
|  618915878|-2125095252|       10|        Por quererte|
| 1200624010|-2125095252|        6|    Solo Dejate Amar|
| -344769466|-2125095252|        6|                What|
|-1092166790|-2125095252|        6|              Besame|
| 1580252322|-2125095252|        5|Guerrilla Monsoon...|
|-1407064031|-2125095252|        5|        Dile al amor|
|-2024781075|-2125095252|        5|The Christmas Son...|
|  623026281|-2125095252|        4|        Fruta Fresca|
| -569916139|-2125095252|        3|   Fique Em Silencio|
|  -14492029|-2125095252|        3|Ojalá que llueva ...|
| -681206166|-2125095252|        2|              Dímelo|
| 1070711413|-2125095252|        1|Un Coche De Agua ...|
| 1293937135|-2125095252|        1|            Waterloo|
| 1280342110|-2125095252|        1|                Niña|
| -100236852|-2125095252|      

In [230]:
df2.limit(10).show()

+------------------+------------------+-----------+---------+-----------+
|              song|          ArtistId|     userId|playCount|        aId|
+------------------+------------------+-----------+---------+-----------+
|SOSDCFG12AB0184647|ARSB5591187B99A848| -818976768|        1|  299552969|
|SOGNNYL12A6D4F910B|ARJ3CTF1187B9A1F2E| -708621849|        1| -960455635|
|SOWWQXA12A81C1F9CA|AR4T2IF1187B9ADBB7|  188613323|        1| -624646109|
|SOHNWIM12A67ADF7D9|ARU3C671187FB3F71B|  224547268|        4|-1610988372|
|SOECFIW12A8C144546|ARPAAPH1187FB3601B|  877169706|        1| 1682736681|
|SOGWEOB12AB018A4D0|ARNNRN31187B9AE7B7| 1162629209|        2| 1537530521|
|SOGWEOB12AB018A4D0|ARNNRN31187B9AE7B7| -297579563|        1| 1537530521|
|SOGWEOB12AB018A4D0|ARNNRN31187B9AE7B7|-1417885395|       13| 1537530521|
|SOUMLGC12A6D4F9C32|AREY11W1187FB36FBD| 2133239319|       13| 1558764962|
|SOJGCRL12A8C144187|AR1FEUF1187B9AF3E3|-1865233319|        1| 1260906859|
+------------------+------------------

In [262]:
path = "unique_artists.txt"
artists = spark.read.csv(path,sep="<SEP>")
artists = artists.withColumnRenamed("_c0","artistId").withColumnRenamed("_c1","AmbId").withColumnRenamed("_c2","tid").withColumnRenamed("_c3","ArtistName")
artists  = artists.withColumn("aId",F.hash("artistId"))
artists.show()

+------------------+--------------------+------------------+--------------------+-----------+
|          artistId|               AmbId|               tid|          ArtistName|        aId|
+------------------+--------------------+------------------+--------------------+-----------+
|AR002UA1187B9A637D|7752a11c-9d8b-422...|TRMUOZE12903CDF721|        The Bristols| -627204226|
|AR003FB1187B994355|1dbd2d7b-64c8-46a...|TRWDPFR128F93594A6|            The Feds| 1343206729|
|AR006821187FB5192B|94fc1228-7032-4fe...|TRMZLJF128F4269EAC|Stephen Varcoe/Ch...|-1476354621|
|AR009211187B989185|9dfe78a6-6d91-454...|TRMGURO12903CAE2F0|    Carroll Thompson|  971141839|
|AR009SZ1187B9A73F4|8cd574c0-b9f7-499...|TRGWWFP12903CE7E79|           Gorodisch|  333413468|
|AR00A1N1187FB484EB|7373764f-c642-439...|TRMMUWQ128F92E88FC|      1.000 Mexicans|-2037147711|
|AR00A6H1187FB5402A|312c14d9-7897-460...|TRWNEQX12903CB84FB|         The Meatmen|-1709521807|
|AR00AP71187B99635F|607d7275-2b92-49e...|TRWXQXG12903CE91BD|

In [263]:
temp3 = artists.select("aId","ArtistId","tid").join(tracks,on="tid")
temp3 = temp3.drop("songId")
#temp3.select("song","ArtistName").show()
temp3.show()

+------------------+-----------+------------------+------------------+--------------------+--------------------+
|               tid|        aId|          ArtistId|              song|          ArtistName|           SongTitle|
+------------------+-----------+------------------+------------------+--------------------+--------------------+
|TRMMMYQ128F932D901|-1991502513|ARYZTJS1187B98C555|SOQMMHC12AB0180CB8|    Faster Pussy cat|        Silent Night|
|TRMMMKD128F425225D|-1587710549|ARMVN3U1187FB3A1EB|SOVFVAK12A8C1350D9|    Karkkiautomaatti|         Tanssi vaan|
|TRMMMRX128F93187D9| 1327595459|ARGEKB01187FB50750|SOGTUKN12AB017F4F1|      Hudson Mohawke|   No One Could Ever|
|TRMMMCH128F425532C|-1259373783|ARNWYLR1187B9B2F9C|SOBNYVR12A8C13558C|         Yerba Brava|       Si Vos Querés|
|TRMMMWA128F426B589| 1769828720|AREQDTE1269FB37231|SOHSBXH12A8C13B0DF|          Der Mystic|    Tangle Of Aspens|
|TRMMMXN128F42936A5|  324773234|AR2NS5Y1187FB5879D|SOZVAPQ12A8C13B63C|    David Montgomery|Symph

In [264]:
temp3 = temp3.join(df2,on="song").select("song","ArtistId","userId","playCount","ArtistName")
temp3.show()

+------------------+------------------+-----------+---------+--------------------+
|              song|          ArtistId|     userId|playCount|          ArtistName|
+------------------+------------------+-----------+---------+--------------------+
|SOSDCFG12AB0184647|ARSB5591187B99A848| -818976768|        1|     Lena Philipsson|
|SOGNNYL12A6D4F910B|ARJ3CTF1187B9A1F2E| -708621849|        1|Prince & The Revo...|
|SOWWQXA12A81C1F9CA|AR4T2IF1187B9ADBB7|  188613323|        1|          Billy Idol|
|SOHNWIM12A67ADF7D9|ARU3C671187FB3F71B|  224547268|        4|           Morcheeba|
|SOECFIW12A8C144546|ARPAAPH1187FB3601B|  877169706|        1|             Bacilos|
|SOGWEOB12AB018A4D0|ARNNRN31187B9AE7B7| 1162629209|        2|            Al Green|
|SOGWEOB12AB018A4D0|ARNNRN31187B9AE7B7| -297579563|        1|            Al Green|
|SOGWEOB12AB018A4D0|ARNNRN31187B9AE7B7|-1417885395|       13|            Al Green|
|SOUMLGC12A6D4F9C32|AREY11W1187FB36FBD| 2133239319|       13|        STEVE TAYLOR|
|SOJ

In [265]:
temp3 = temp3.groupby(["ArtistId","userId"]).sum("playCount").sort("sum(playCount)",ascending=False)
temp3= temp3.withColumn("aId", F.hash(col("ArtistId")))

In [266]:
artistRecommendationData = temp3

In [267]:
(training, test) = artistRecommendationData.randomSplit([0.7, 0.3],seed = 20)

# rank=20,maxIter=10, regParam=0.01,
als = ALS(
          userCol="userId", itemCol="aId", ratingCol="sum(playCount)",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)
# Import the requisite packages
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 15, 25, 100]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse",
           labelCol="sum(playCount)",
           predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
#Fit cross validator to the 'train' dataset
model = cv.fit(training)
#Extract best model from the cv model above
best_model = model.bestModel
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

Num models to be tested:  16
22/11/13 23:04:57 WARN DAGScheduler: Broadcasting large task binary with size 1006.2 KiB
22/11/13 23:04:59 WARN DAGScheduler: Broadcasting large task binary with size 1006.2 KiB
22/11/13 23:05:01 WARN DAGScheduler: Broadcasting large task binary with size 1006.2 KiB
22/11/13 23:05:03 WARN DAGScheduler: Broadcasting large task binary with size 1006.2 KiB
22/11/13 23:05:23 WARN DAGScheduler: Broadcasting large task binary with size 1006.3 KiB
22/11/13 23:05:25 WARN DAGScheduler: Broadcasting large task binary with size 1006.3 KiB
22/11/13 23:05:27 WARN DAGScheduler: Broadcasting large task binary with size 1006.3 KiB
22/11/13 23:05:28 WARN DAGScheduler: Broadcasting large task binary with size 1006.3 KiB
22/11/13 23:05:48 WARN DAGScheduler: Broadcasting large task binary with size 1006.2 KiB
22/11/13 23:05:50 WARN DAGScheduler: Broadcasting large task binary with size 1006.2 KiB
22/11/13 23:05:52 WARN DAGScheduler: Broadcasting large task binary with size 100

In [268]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)
# Generate n Recommendations for all users
recommendations = best_model.recommendForAllUsers(5)
recommendations.show()



5.234384078978725


+-----------+--------------------+
|     userId|     recommendations|
+-----------+--------------------+
|-2130942721|[{1168512310, 0.9...|
|-2100266772|[{-557639096, 0.5...|
|-2078707018|[{-1007996970, 0....|
|-2067855073|[{960946619, 1.09...|
|-2058327330|[{1746977676, 1.0...|
|-2046419562|[{-31512273, 1.50...|
|-2037466548|[{1918241559, 0.4...|
|-2016827684|[{891287066, 5.73...|
|-1995191281|[{-1925251021, 0....|
|-1983755226|[{-1007996970, 1....|
|-1964357835|[{-1925251021, 1....|
|-1959900652|[{-474144814, 0.6...|
|-1945625912|[{-1007996970, 0....|
|-1933481207|[{316087039, 1.05...|
|-1925532506|[{1575769368, 7.8...|
|-1915006186|[{-557639096, 3.4...|
|-1852287404|[{-78443126, 0.53...|
|-1845661126|[{1478580568, 0.0...|
|-1843921463|[{1746977676, 0.8...|
|-1843310532|[{-557639096, 2.6...|
+-----------+--------------------+
only showing top 20 rows



In [269]:
from pyspark.sql.functions import explode
nrecommendations = recommendations\
    .withColumn("rec_exp",  explode("recommendations"))\
    .select('userId', col("rec_exp.aId"), col("rec_exp.rating"))
nrecommendations.limit(10).show()

+-----------+-----------+----------+
|     userId|        aId|    rating|
+-----------+-----------+----------+
|-2130942721| 1168512310| 0.9124969|
|-2130942721|  689114149| 0.8111042|
|-2130942721| -813236727|0.76041245|
|-2130942721|  891287066| 0.7055794|
|-2130942721|  229267030| 0.7019551|
|-2100266772| -557639096| 0.5822251|
|-2100266772|-1925251021|0.37411895|
|-2100266772|  891287066|0.26224357|
|-2100266772|-1198101392|0.25049472|
|-2100266772| 1070782056|0.20004876|
+-----------+-----------+----------+



In [286]:
nrecommendations.join(artists.select("aId","ArtistId","ArtistName"), on='aId').filter('userId = -2130942721').sort('rating',ascending=False).select("userId","rating","ArtistId","ArtistName").show()

+-----------+----------+------------------+-------------+
|     userId|    rating|          ArtistId|   ArtistName|
+-----------+----------+------------------+-------------+
|-2130942721| 0.9124969|ARHAEXZ1187FB54F3C|     N.E.R.D.|
|-2130942721| 0.8111042|ARBCZ031187B9B9CB9|     Colossal|
|-2130942721|0.76041245|ARUJZFJ1187B9B135F| Shania Twain|
|-2130942721| 0.7055794|ARFCWSZ123526A0AFD|Justin Bieber|
|-2130942721| 0.7019551|AREHKZU122E5C4FC81|  Mike Posner|
+-----------+----------+------------------+-------------+



In [287]:
temp3.join(artists,on="ArtistId").filter('userId = -2130942721').sort('sum(playCount)', ascending=False).select("ArtistId","userId","sum(playCount)","ArtistName",).limit(20).show()

+------------------+-----------+--------------+---------------+
|          ArtistId|     userId|sum(playCount)|     ArtistName|
+------------------+-----------+--------------+---------------+
|AREHKZU122E5C4FC81|-2130942721|             7|    Mike Posner|
|ARBCZ031187B9B9CB9|-2130942721|             6|       Colossal|
|ARZO9UQ1187FB4D261|-2130942721|             3|Alliance Ethnik|
|ARUJZFJ1187B9B135F|-2130942721|             2|   Shania Twain|
|ARHAEXZ1187FB54F3C|-2130942721|             1|       N.E.R.D.|
|ARNSECX11E2835DB52|-2130942721|             1|  Amity in fame|
|ARFCWSZ123526A0AFD|-2130942721|             1|  Justin Bieber|
+------------------+-----------+--------------+---------------+

